In [2]:
import numpy as np

In [18]:
def row_from_bitstring(bs) :
    A = np.array([int(bs[i]) for i in range(0,len(bs))])
    A.reshape(1,-1)
    return A

def bitstring_from_row(row) :
    return ''.join([str(x[0]) for x in row])

        
valid_rows = []
keys = []

for i in range(0,2**(8)) :
    bs = format(i, '08b')
    A = row_from_bitstring(bs)
    if (np.sum(A, dtype=np.int32) % 2) == 1 : 
        valid_rows.append((bs,A))
        keys.append(bs)
    
def possible_rows(row, candidates) :
    return [cand for cand in candidates if row[1] @ (cand[1]).T % 2 ==  0]

goes_with_row = {}

for row in valid_rows :
    goes_with_row[row[0]] = possible_rows(row, valid_rows) 

print(set([len(goes_with_row[bs]) for bs in keys]))

unitaries = []

def build_matrixes(size) :
    for bs in keys :
        build_next_row(row_from_bitstring(bs), set(goes_with_row[bs]))

def build_next_row(so_far, candidate_rows) :
    if len(so_far.shape) > 1 and so_far.shape[1] == 8 :
        unitaries.append(so_far)
        print(so_far)
    else :
        for row in candidate_rows :
            new_candidates = candidate_rows.intersect(goes_with_row[bitstring_from_row(row)])
            build_next_row(np.vstack(so_far, row), new_candidates)
        
build_matrixes(8)

{64}


TypeError: unhashable type: 'numpy.ndarray'

In [20]:
#####
# This function takes a string of zeros and ones, 
# converts them to integers, computes the sum
# then returns true if the sum is odd, false if it is even
##
# The idea is that the string of zeros and ones can appear in a unitary matrix U over Z/2
# as the diagonal entry of UU* represented by this row will be 1
def is_valid_row(bs) :
    row_sum = sum([int(bs[i]) for i in range(0,len(bs))])
    return ((row_sum % 2) == 1)

#####
# This function takes two string of zeros and ones, 
# assumed of the same length 
# Computes their dot product as vectors and computes the sum of the resulting vector
# then returns true if the sum is even, false if it is odd
##
# THe idea is that these two bitstrings can appear as rows in the same unitary matrix U over Z/2
# as the off diagonal entry of UU* they represent will be 0
def can_coexist(bsx, bsy) :
    product_sum = sum([int(bsx[i])*int(bsy[i]) for i in range(0,len(bsx))])
    return ((product_sum % 2) == 0)

## Build up valid rows
def get_valid_rows(n) :
    valid_rows = []
    for i in range(0,2**(n)) :
        bs = format(i, '0'+str(n)+'b')
        if is_valid_row(bs) : 
            valid_rows.append(bs)
            
    return valid_rows

### Make a dictionary, with each row as a key
# The value of that key is a list of the rows it can match with
def get_row_dict(rows) :
    row_dict = {}
    for row in rows :
        row_dict[row] = [y for y in rows if can_coexist(row, y)]
    
    return row_dict


def add_row(so_far, candidates, remains) :
    if remains == 0 :
        unitaries.append(so_far)
    elif len(candidates) > 0 :
        for row in candidates :
            new_candidates = [cand for cand in candidates if cand in matching_dict[row]]
            add_row(so_far + [row], new_candidates, remains-1)
                


print("Building rows and dictionary...")
valid_rows = get_valid_rows(8)
matching_dict = get_row_dict(valid_rows)
print("Done.")
print("")
unitaries = []
print("Finding unitaries")

###### Status printing variables
total_rows = len(valid_rows)
count = 0

for row in valid_rows :
    add_row([row], matching_dict[row], 7)
    
    ### Status printing 
    count = count + 1
    if count % (total_rows // 10) == 0 :
        print("We have completed %i rows" % count)
    
print(len(unitaries))

Building rows and dictionary...
Done.

Finding unitaries
We have completed 12 rows
We have completed 24 rows
We have completed 36 rows
We have completed 48 rows
We have completed 60 rows
We have completed 72 rows


MemoryError: 